In [5]:
import re
import numpy as np
import pandas as pd
import pymorphy2
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import LSTM, Dropout, GRU, Embedding, TimeDistributed
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer 
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import random
import sys
from difflib import SequenceMatcher
import operator
import gensim
from nltk import tokenize
from rifmator_engine import Rifmator

C:\Users\asigachev\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\asigachev\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


ModuleNotFoundError: No module named 'rifmator_engine'

# Модель

In [43]:
class LyricsGenModel:
    def __init__(self):
        self.text_corp = None
        self.tokens = None
        self.vocab_size = None # Размер словаря - в итоге должен определиться из материала
        self.seq_length = None # Длина последовательности слов, на которой обучаемся
        self.model = None # Основная модель, по которой генерируется текст
        self.generated_flow = None
        self.length = None
    
    def set_length(self, value):
        self.length = value
    
    def load_data(self, data_file_name):
        # Функция загружает тексты песен из Pandas Dataframe
        print('Загрузка данных')
        df = pd.read_csv(data_file_name, encoding='cp1251')
        songs_lyrics = list(df['0'])
        print('Найдено песен:', len(songs_lyrics))
        
        self.text_corp = ' '.join(songs_lyrics)    # Создаём общий корпус текста - объединение всех песен 
        
        # Удаляем все спецсимволы и разделяем склеившиеся слова
        remove_punct = re.compile('[^а-яА-Я\s\dё]') 
        self.text_corp = re.sub(remove_punct, '', self.text_corp) 
        garbled_words_regex = re.compile('(?P<A>[А-я][а-я]+)(?P<B>[А-Я][а-я]*)')
        self.text_corp = re.sub(garbled_words_regex, '\g<A> \g<B>', self.text_corp)
        
        # Токенизация - разделение текста на слова
        self.tokens = self.text_corp.lower().split()
        print('Получено токенов из корпуса текста:', len(self.tokens))
        
        # Получение последовательностей из N токенов. По первым токенам, потом будем предсказывать последний
        self.sequences = list()
        for i in range(self.length, len(self.tokens)):
            # select sequence of tokens
            seq = self.tokens[i-self.length:i]
            # convert into a line
            line = ' '.join(seq)
            # store
            self.sequences.append(line)
        print(self.sequences[:10])    
        self.tokenizer = Tokenizer()
        self.tokenizer.fit_on_texts(self.sequences)
        self.vocab_size = len(self.tokenizer.word_index) + 1
        
        print('Размер словаря:', self.vocab_size)

        sequences_enc = self.tokenizer.texts_to_sequences(self.sequences)
        sequences_enc = np.array(sequences_enc)        
        self.X, self.y = sequences_enc[:,:-1], sequences_enc[:,-1]
        self.seq_length = self.X.shape[1]
        
        # Обучение word2vec-модели для выбора рифм
        
        self.w2v_model = gensim.models.Word2Vec([self.tokens], size=100, window=5, min_count=1, workers=4)
       
        
    def build_model(self):
        model = Sequential()
        model.add(Embedding(self.vocab_size, 50, input_length=self.seq_length))
        model.add(GRU(100, return_sequences=True))
        model.add(GRU(100))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(self.vocab_size, activation='softmax'))

        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        self.model = model
        
        print('Модель создана:')
        print(model.summary())
        
    def train_model(self, b_size, ep_num):
        self.model.fit(self.X, self.y, batch_size=b_size, epochs=ep_num) 
    
    def save_model(self, file_name):
        self.model.save(file_name)
    
    def save_weights(self, file_name):
        self.model.save_weights(file_name)

    def load_model(self, file_name):
        self.model = load_model(file_name)
        print(self.model.summary())
    
    def load_weights(self, file_name):
        self.model.load_weights(file_name)
        
    def get_seed_text(self):
        self.seed_text = self.sequences[random.randint(0,len(self.sequences))]

    def sample(self, preds, diversity):
        # Функция принимает список индексов и соответствующих им вероятностей
        # Нормализует вероятности так, чтобы их сумма была равной 1
        # Возвращает индекс с соответствующей ему вероятностью
        indexes = preds.argsort()[0][-diversity:]
        preds.shape
        probs = [float(preds[:,x]) for x in indexes]
        print('probabilities:', ['%.2f' % elem for elem in probs])
        normalized_probs = [x/sum(probs) for x in probs]
        print('normalized_probabilities', ['%.2f' % elem for elem in normalized_probs])
        return np.random.choice(indexes, p=normalized_probs)        

    def generate_seq(self, n_words):
        result = list()
        in_text = self.seed_text
        # Создаём поток из n_words слов
        for _ in range(n_words):
            # Кодируем сидирующую последовательность в числа, чтобы подать на вход модели
            encoded = self.tokenizer.texts_to_sequences([in_text])[0]
            
            # Приведение последовательности к необходимой длине
            encoded = pad_sequences([encoded], maxlen=self.seq_length, truncating='pre')
            
            # Предсказываем вероятности для вариантов следующего слова
            pred = self.model.predict_proba(encoded, verbose=0)
            
            yhat = np.argmax(pred)
            
            # Восстанавливаем слово по целочисленному значению
            out_word = ''
            for word, index in self.tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
                    
            # Приделываем слово к потоку
            in_text += ' ' + out_word
            result.append(out_word)
        print('Результат сохранён в атрибут generated_flow')
        self.generated_flow = ' '.join(result)    
        
    def generate_seq_with_randomness(self, n_words, diversity):
        result = list()
        in_text = self.seed_text
        # Создаём поток из n_words слов
        for _ in range(n_words):
            # Кодируем сидирующую последовательность в числа, чтобы подать на вход модели
            encoded = self.tokenizer.texts_to_sequences([in_text])[0]
            
            # Приведение последовательности к необходимой длине
            encoded = pad_sequences([encoded], maxlen=self.seq_length, truncating='pre')
            # Предсказываем вероятности для вариантов следующего слова
            pred = self.model.predict_proba(encoded, verbose=0)
            
            yhat = self.sample(pred, diversity)

            # Восстанавливаем слово по целочисленному значению
            out_word = ''
            for word, index in self.tokenizer.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            # append to input
            in_text += ' ' + out_word
            result.append(out_word)
        print('Результат сохранён в атрибут generated_flow')
        self.generated_flow = ' '.join(result)      

    def get_verses(self, text):
        # Функция получает пару рифмованных строк
        max_verse_len = 6
        min_verse_len = 1
        rhyme_threshold = 0.935        
        flow_list = text.split()
        long_words= [x for x in set(self.tokens) if len(x)>2]
        
        for i in range(max_verse_len,-1,-1):
            for j in range(i+1, i+1+max_verse_len):
                #print(flow_list[i], flow_list[j], self.rhyme_score(flow_list[i],flow_list[j], False))
                if self.rhyme_score(flow_list[i],flow_list[j], False) > rhyme_threshold:
                    verse_a = ' '.join(flow_list[:i+1])
                    verse_b = ' '.join(flow_list[i+1:j+1])
                    return verse_a, verse_b, verse_b
        else:       
            
            
            for i in range(max_verse_len,-1,-1):
                word_to_put = None
                if len(flow_list[i]) < 3:
                    continue
                else:
                    word_to_rhyme = flow_list[i]
                # Список индексов слов в случайном порядке. Будем пытаться найти рифму к словам из этого списка 
                clist = random.sample([x for x in range(min_verse_len,max_verse_len+1)], max_verse_len-min_verse_len+1)
                
                
        
                for c in clist:
                    word_to_change = flow_list[i+c] # Получаем слово, которое будем менять
                    #print('Пробуем искать рифмы к слову ', word_to_rhyme)
                    rhymes_found = self.find_best_rhymes(word_to_rhyme,  long_words, rhyme_threshold, word_to_change) #Ищем лучшие рифмы к слову
                    #print('Найдено рифм', len(rhymes_found))
                    if len(rhymes_found) > 0: #Если рифма нашлась
                        word_to_put = rhymes_found[random.randint(1,len(rhymes_found))-1] #Выбираем слово, которое вставляем
                        break
                if word_to_put == None:
                    continue
                #print('Меняем', word_to_change, 'на', word_to_put)
                verse_a = text.split(word_to_rhyme)[0]+word_to_rhyme
                #print('verse_a', verse_a)
                verse_b_orig = text.replace(verse_a+' ', '').split(word_to_change)[0] + word_to_change
                #print('verse_b_orig', verse_b_orig)
                verse_b_aslist = verse_b_orig.split()
                verse_b_aslist[-1] = word_to_put
                verse_b = ' '.join(verse_b_aslist)
                return verse_a, verse_b, verse_b_orig

    def create_verse(self, max_line_len):
        # Функция формирует стих
        flow = self.generated_flow
        verses = list()
        self.verses_final = list()
        print(len(flow.split()))
        while len(flow.split()) > max_line_len*2+1:
            new_verses = self.get_verses(flow)
            verses.extend(new_verses)
            self.verses_final.extend(new_verses[:-1])
            flow = flow.split(verses[-1]+' ')[-1]
            print(len(flow))
        
    def similar(self, a, b):
        return SequenceMatcher(None, a, b).ratio()
    
    def rhyme_score(self, a, b, verbose):
        score = 0
        
        # Слова должны быть длиннее 3 символов, для всего, что короче возвращаем 0
        if len(a) < 3 or len(b) < 3:
            return 0
        
        # Для одинаковых слов возвращать 0
        
        if a == b:
            return 0
        
        maxlen = min(len(a),len(b), 4)+1
        for i in range(1,maxlen):
            score += self.similar(a[-i:], b[-i:])
        score = score / (maxlen - 1)   
        return score
    
    def find_best_rhymes(self, word_to_check, words_list, threshold, word_to_change=''):
        if len(word_to_check) < 3:
            return []
        words_set = set(words_list)
        potential_rhymes = list()
        similarity = 0
        for word in words_set:
            if self.rhyme_score(word_to_check, word, False) > threshold and word not in word_to_check and word_to_check not in word:# здесь поменять на леммы
                potential_rhymes.append(word)
                similarity = self.rhyme_score(word_to_check, word, False)

        # Если указано слово, которое меняем фильтруем рифмы по близости к слову, которое меняем
        if word_to_change !='':
            potential_rhymes = self.find_best_rhymes(word_to_check, words_set, threshold)
            rhyme_proximity_dict = {}
            for i in potential_rhymes:
                rhyme_proximity_dict.update({i : self.w2v_model.wv.similarity(word_to_change, i.lower())})
            chosen_rhymes = sorted(rhyme_proximity_dict.items(),
                                   key=operator.itemgetter(1), reverse=True)[:1+min(len(rhyme_proximity_dict)//3, 
                                                                                    len(rhyme_proximity_dict))]
            return [x[0] for x in chosen_rhymes]
        # Если не указано, просто возвращаем список найденных рифм
        else:
            return potential_rhymes


    def print_verses(self):
        for verse in self.verses_final:
            print(verse)

# Обучение на хип-хоп-текстах

In [34]:
mymodel_hiphop = LyricsGenModel() #Создание объекта

In [35]:
mymodel_hiphop.set_length(4) #Установка длины последовательности

In [36]:
mymodel_hiphop.load_data('lyrics-extended.csv') #Загрузка данных

Загрузка данных
Найдено песен: 956
Получено токенов из корпуса текста: 311707
['это мне поручили узнать', 'мне поручили узнать куда', 'поручили узнать куда время', 'узнать куда время течет', 'куда время течет и', 'время течет и когда', 'течет и когда в', 'и когда в истории', 'когда в истории начнется', 'в истории начнется новый']
Размер словаря: 51777


In [37]:
mymodel_hiphop.load_model('loss15extendedlyr') #Загрузка предобученной модели

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 3, 50)             2588850   
_________________________________________________________________
lstm_19 (LSTM)               (None, 3, 100)            60400     
_________________________________________________________________
lstm_20 (LSTM)               (None, 100)               80400     
_________________________________________________________________
dense_19 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_20 (Dense)             (None, 51777)             5229477   
Total params: 7,969,227
Trainable params: 7,969,227
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
mymodel_hiphop.load_weights('loss15extendedlyr-w') #Загрузка весов

In [39]:
mymodel_hiphop.get_seed_text() #Получение стартовой последовательности

In [40]:
mymodel_hiphop.generate_seq_with_randomness(200,3) #Генерация потока слов 
#В параметрах длина потока и число наиболее вероятных слов, из которого выбирается следующее слово

probabilities: ['0.05', '0.09', '0.18']
normalized_probabilities ['0.15', '0.29', '0.57']
probabilities: ['0.08', '0.11', '0.57']
normalized_probabilities ['0.10', '0.14', '0.75']
probabilities: ['0.03', '0.05', '0.90']
normalized_probabilities ['0.03', '0.05', '0.92']
probabilities: ['0.10', '0.11', '0.16']
normalized_probabilities ['0.27', '0.30', '0.43']
probabilities: ['0.06', '0.07', '0.85']
normalized_probabilities ['0.06', '0.07', '0.87']
probabilities: ['0.00', '0.00', '1.00']
normalized_probabilities ['0.00', '0.00', '1.00']
probabilities: ['0.03', '0.10', '0.83']
normalized_probabilities ['0.03', '0.10', '0.87']
probabilities: ['0.03', '0.04', '0.92']
normalized_probabilities ['0.03', '0.04', '0.93']
probabilities: ['0.05', '0.08', '0.77']
normalized_probabilities ['0.05', '0.09', '0.86']
probabilities: ['0.01', '0.08', '0.88']
normalized_probabilities ['0.01', '0.08', '0.90']
probabilities: ['0.00', '0.00', '1.00']
normalized_probabilities ['0.00', '0.00', '1.00']
probabilit

probabilities: ['0.17', '0.17', '0.19']
normalized_probabilities ['0.32', '0.32', '0.36']
probabilities: ['0.12', '0.23', '0.47']
normalized_probabilities ['0.15', '0.28', '0.58']
probabilities: ['0.18', '0.25', '0.29']
normalized_probabilities ['0.25', '0.34', '0.40']
probabilities: ['0.00', '0.00', '1.00']
normalized_probabilities ['0.00', '0.00', '1.00']
probabilities: ['0.01', '0.01', '0.01']
normalized_probabilities ['0.28', '0.34', '0.39']
probabilities: ['0.02', '0.03', '0.94']
normalized_probabilities ['0.02', '0.03', '0.95']
probabilities: ['0.07', '0.12', '0.71']
normalized_probabilities ['0.08', '0.13', '0.79']
probabilities: ['0.08', '0.12', '0.36']
normalized_probabilities ['0.15', '0.21', '0.64']
probabilities: ['0.08', '0.11', '0.13']
normalized_probabilities ['0.24', '0.35', '0.41']
probabilities: ['0.11', '0.14', '0.19']
normalized_probabilities ['0.24', '0.32', '0.44']
probabilities: ['0.10', '0.10', '0.34']
normalized_probabilities ['0.19', '0.19', '0.62']
probabilit

In [41]:
mymodel_hiphop.generated_flow #Сгенерированный текст

'радио тренировочные мясо рекламу расширила как щелкаю бро раком густая падла они культуру просторы плавится с кто века знакомый откройте глаза страна модели испытаний забирает спасибо глаз и я не знаю мэн именно ты всегда слушаешь и че там занимайте в каменном ноги смерть еее знаний подписка где смешалось мы тонем пополам разделения и мигают пять отсчёт солнце просты на потери так закрою путь это про укурку это осталось сказал лживые денег и любит джаз секунду что меньше головой это удар скандалам послушай ак47 я с собой на ковчег остаться человеком среди сволочей в душ ярко тюбик трава и отрабатывать у ставил те кто внезапно не пускай даже не скрывают на встречу такие в одним небом процесс кемто всех аккуратней не осталось ни чего ты не может это не комильфо или моветон и ктото экономил на дом а я сделал пришло итак сон но я без тебя я не могу жить я начал примером брать хватит надо не выпустят отсюда он не приедет в харьков да кстати кто ты скажешь как соседи стоит ночь в главной ми

In [42]:
mymodel_hiphop.create_verse(6) #Добавление рифм и разбиение на строки

200
Пробуем искать рифмы к слову  щелкаю
Найдено рифм 8
1067
Пробуем искать рифмы к слову  плавится
Найдено рифм 263
1017
Пробуем искать рифмы к слову  испытаний
Найдено рифм 44
942
Пробуем искать рифмы к слову  именно
Найдено рифм 28
895
Пробуем искать рифмы к слову  еее
Найдено рифм 0
Пробуем искать рифмы к слову  еее
Найдено рифм 0
Пробуем искать рифмы к слову  еее
Найдено рифм 0
Пробуем искать рифмы к слову  еее
Найдено рифм 0
Пробуем искать рифмы к слову  еее
Найдено рифм 0
Пробуем искать рифмы к слову  еее
Найдено рифм 0
Пробуем искать рифмы к слову  смерть
Найдено рифм 1
821
Пробуем искать рифмы к слову  отсчёт
Найдено рифм 3
747
Пробуем искать рифмы к слову  осталось
Найдено рифм 38
679
Пробуем искать рифмы к слову  удар
Найдено рифм 2
628
Пробуем искать рифмы к слову  ковчег
Найдено рифм 1
569
Пробуем искать рифмы к слову  трава
Найдено рифм 8
522
Пробуем искать рифмы к слову  пускай
Найдено рифм 7
479
Пробуем искать рифмы к слову  небом
Найдено рифм 6
420
Пробуем искать рифмы

In [44]:
mymodel_hiphop.print_verses() #Вывод стиха

радио тренировочные мясо рекламу расширила как щелкаю
бро втыкаю
густая падла они культуру просторы плавится
с палится
века знакомый откройте глаза страна модели испытаний
забирает спасибо гуляний
и я не знаю мэн именно
ты всегда слушаешь и мгновенно
там занимайте в каменном ноги смерть
еее знаний подписка где смешалось круговерть
тонем пополам разделения и мигают пять отсчёт
солнце просты на потери учёт
закрою путь это про укурку это осталось
сказал лживые денег и разлетелось
джаз секунду что меньше головой это удар
краснодар
послушай ак47 я с собой на ковчег
остаться человеком витчег
сволочей в душ ярко тюбик трава
и удава
у ставил те кто внезапно не пускай
даже напичкай
скрывают на встречу такие в одним небом
процесс кемто ознобом
аккуратней не осталось ни чего
ты соседнего
может это не комильфо или моветон
брайтон


# Обучение на детских стихах

In [ ]:
mymodel_hiphop.train_model(128,ep_num=15)

In [ ]:
mymodel_hiphop.model.save('loss15extendedlyr')

In [ ]:
mymodel_hiphop.save_weights('loss15extendedlyr-w')

In [45]:
mymodel_child_poetry = LyricsGenModel()

In [46]:
mymodel_child_poetry.set_length(3)

In [47]:
mymodel_child_poetry.load_data('stihi-ext.csv')

Загрузка данных
Найдено песен: 83009
Получено токенов из корпуса текста: 83009
['раздватричетырепять будем пальчики', 'будем пальчики считать', 'пальчики считать крепкие', 'считать крепкие дружные', 'крепкие дружные все', 'дружные все такие', 'все такие нужные', 'такие нужные на', 'нужные на другой', 'на другой руке']
Размер словаря: 18512


In [53]:
mymodel_child_poetry.load_model('ch-model-seq3')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2, 50)             925600    
_________________________________________________________________
gru_3 (GRU)                  (None, 2, 100)            45300     
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 18512)             1869712   
Total params: 2,911,012
Trainable params: 2,911,012
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
mymodel_child_poetry.load_weights('ch-weights-seq3')

In [55]:
mymodel_child_poetry.get_seed_text()

In [56]:
mymodel_child_poetry.generate_seq_with_randomness(100,2)

probabilities: ['0.08', '0.27']
normalized_probabilities ['0.23', '0.77']
probabilities: ['0.16', '0.61']
normalized_probabilities ['0.21', '0.79']
probabilities: ['0.20', '0.77']
normalized_probabilities ['0.21', '0.79']
probabilities: ['0.13', '0.33']
normalized_probabilities ['0.28', '0.72']
probabilities: ['0.12', '0.38']
normalized_probabilities ['0.24', '0.76']
probabilities: ['0.06', '0.65']
normalized_probabilities ['0.09', '0.91']
probabilities: ['0.23', '0.25']
normalized_probabilities ['0.48', '0.52']
probabilities: ['0.16', '0.17']
normalized_probabilities ['0.49', '0.51']
probabilities: ['0.12', '0.12']
normalized_probabilities ['0.50', '0.50']
probabilities: ['0.06', '0.09']
normalized_probabilities ['0.38', '0.62']
probabilities: ['0.03', '0.87']
normalized_probabilities ['0.04', '0.96']
probabilities: ['0.01', '0.97']
normalized_probabilities ['0.01', '0.99']
probabilities: ['0.03', '0.90']
normalized_probabilities ['0.04', '0.96']
probabilities: ['0.07', '0.07']
normal

In [57]:
mymodel_child_poetry.generated_flow

'другая тридцать лет работать подшефный петю здесь как будто месяц май а паренёк раздался класс со взрослой лягушки же пыли напишу ликуют в третьем ура мы победили кстати ли некстати ли скажу начистоту такие показатели грязнят всю чистоту не поверила а мальчишки в путь не по дням по свету возвратятся в книжку вести от читателей от вас принимается танцуют и и одно только слово твердит айболит лимпопо лимпопо лимпопо и акула каракула распахнула злую пасть вы в конце концов взмолился андрей петрович опоздаем торопят артисты борисов достаёт не пьёт он в ответ в кровать предлагает день начинаются света лида это'

In [58]:
mymodel_child_poetry.create_verse(6)

100
583
583
532
465
430
374
332
280
210
166
86
49


In [59]:
mymodel_child_poetry.print_verses()

другая тридцать
лет работать
подшефный петю здесь
проснитесь
подшефный петю здесь
как будто месяц май а смейтесь
раздался класс со взрослой лягушки же пыли
напишу ликуют в жилибыли
ура мы победили кстати
ли некстати
ли скажу начистоту
такие показатели грязнят всю чистоту
не поверила а мальчишки в путь
не по тронуть
по свету возвратятся в книжку вести
пусти
вас принимается танцуют и и одно только
слово твердит айболит тихонько
лимпопо лимпопо и акула каракула
распахнула
злую пасть вы в конце концов взмолился
андрей петрович опоздаем торопят поправился
борисов достаёт не пьёт он в ответ
цвет


In [146]:
mymodel_child_poetry.build_model()

Модель создана:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2, 50)             925600    
_________________________________________________________________
gru_3 (GRU)                  (None, 2, 100)            45300     
_________________________________________________________________
gru_4 (GRU)                  (None, 100)               60300     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_6 (Dense)              (None, 18512)             1869712   
Total params: 2,911,012
Trainable params: 2,911,012
Non-trainable params: 0
_________________________________________________________________
None


In [153]:
mymodel_ch.train_model(b_size=128,ep_num=20)

Epoch 1/20
83006/83006 [==============================] - 182s 2ms/step - loss: 2.6887 - acc: 0.4648
Epoch 2/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.5765 - acc: 0.4833
Epoch 3/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.4881 - acc: 0.4973
Epoch 4/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.4033 - acc: 0.5104
Epoch 5/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.3323 - acc: 0.5197
Epoch 6/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.2637 - acc: 0.5315
Epoch 7/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.2002 - acc: 0.5443
Epoch 8/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.1404 - acc: 0.5501
Epoch 9/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.0862 - acc: 0.5604
Epoch 10/20
83006/83006 [==============================] - 183s 2ms/step - loss: 2.0342 - a

In [154]:
mymodel_ch.save_model('ch-model-seq3')

In [155]:
mymodel_ch.save_weights('ch-weights-seq3')